# 이미지 그리기

In [1]:
def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
    canvas_html = f'''
        <style>
            button#saveButton {{
                background-color: #4CAF50;
                border: none;
                color: white;
                padding: 15px 32px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 16px;
                margin: 4px 2px;
                cursor: pointer;
            }}
        </style>
        <img id="myImage" src="{imgm}" style="display:none;">
        <canvas id="myCanvas" width="{w}" height="{h}"></canvas>
        <canvas id="myCanvas2" width="{w}" height="{h}" style="display:none;"></canvas>
        <button id="saveButton" onclick="saveImage()">Save</button>

        <script>
            var canvas = document.getElementById("myCanvas");
            var ctx = canvas.getContext("2d");
            var isDrawing = false;

            var image = document.getElementById("myImage");
            image.onload = function() {{
                ctx.drawImage(image, 0, 0, {w}, {h});
            }};

            var canvas2 = document.getElementById("myCanvas2");
            var ctx2 = canvas2.getContext("2d");

            canvas.addEventListener("mousedown", startDrawing);
            canvas.addEventListener("mousemove", draw);
            canvas.addEventListener("mouseup", stopDrawing);
            canvas.addEventListener("mouseout", stopDrawing);

            function startDrawing(e) {{
                isDrawing = true;
                draw(e);
            }}

            function draw(e) {{
                if (!isDrawing) return;

                var rect = canvas.getBoundingClientRect();
                var x = e.clientX - rect.left;
                var y = e.clientY - rect.top;

                ctx.strokeStyle = "red";
                ctx.lineWidth = {line_width};
                ctx.lineTo(x, y);
                ctx.stroke();

                ctx2.strokeStyle = "red";
                ctx2.lineWidth = {line_width};
                ctx2.lineTo(x, y);
                ctx2.stroke();
            }}

            function stopDrawing() {{
                isDrawing = false;
                ctx.beginPath();
                ctx2.beginPath();
            }}

            function saveImage() {{
                var image = canvas2.toDataURL("image/png");
                var a = document.createElement("a");
                a.href = image;
                a.download = "{filename}";
                document.body.appendChild(a);
                a.click();
                document.body.removeChild(a);
            }}
        </script>
    '''


    display(HTML(canvas_html))


In [24]:
fname = 'convert/image (1).png'

In [25]:
from IPython.display import display, HTML

import base64, os
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np

image64 = base64.b64encode(open(fname, 'rb').read())
image64 = image64.decode('utf-8')

print(f'Will use {fname} for inpainting')
img = np.array(plt.imread(f'{fname}')[:,:,:3])

Will use convert/image (2-1).png for inpainting


In [26]:
draw(fname, filename=f"./{fname.split('.')[1]}_mask.png", w=img.shape[1], h=img.shape[0], line_width=0.02*img.shape[1])

# LaMa 준비

In [8]:
# apt-get install -y libgl1-mesa-glx

from simple_lama_inpainting import SimpleLama
from PIL import Image

In [9]:
simple_lama = SimpleLama()

# LaMa 실행

In [27]:
mask_path = 'png_mask.png'

image = Image.open(fname)
mask = Image.open(mask_path).convert('L')

In [28]:
result = simple_lama(image, mask)
result.save("inpainted.png")